In [2]:
from forecasting_models.models import *
from forecasting_models.models_2D import *
from tools import *

dropout = 0.03
in_dim = 100
in_dim_2D = 52
device = torch.device('cpu')
k_days = 7
act_func = 'relu'

In [12]:
path = Path('final/firepoint/2x2/train/check_z-score/full_0_10_100/baseline/xgboost_grid_search/')

model = read_object('xgboost_grid_search.pkl', path)
model.get_params()['model'].get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': 0.8,
 'colsample_bynode': None,
 'colsample_bytree': 0.8,
 'device': None,
 'early_stopping_rounds': 15,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.01,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': 1.0,
 'max_depth': 10,
 'max_leaves': None,
 'min_child_weight': 5,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 10000,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 42,
 'reg_alpha': 0.9,
 'reg_lambda': 5.5,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.5,
 'tree_method': 'hist',
 'validate_parameters': None,
 'verbosity': 0}

In [2]:
dico_model = {'GAT': GAT(in_dim=[in_dim, 64, 64, 64],
                heads=[4, 4, 2],
                dropout=dropout,
                bias=True,
                device=device,
                act_func=act_func,
                n_sequences=k_days),

            'ST-GATTCN': STGATCN(n_sequences=k_days,
                                num_of_layers=3,
                                in_channels=in_dim,
                                    end_channels=64,
                                    skip_channels=32,
                                    residual_channels=32,
                                    dilation_channels=32,
                                    dropout=dropout,
                                    heads=6, act_func=act_func,
                                    device=device),
                                
            'ST-GATCONV': STGATCONV(k_days,
                                    num_of_layers=3,
                                    in_channels=in_dim,
                                    hidden_channels=32,
                                    residual_channels=64,
                                    end_channels=32,
                                    dropout=dropout,
                                    heads=6,
                                    act_func=act_func,
                                    device=device),

            'ST-GCNCONV' : STGCNCONV(k_days,
                                    num_of_layers=3,
                                    in_channels=in_dim,
                                    hidden_channels=64,
                                    residual_channels=64,
                                    end_channels=32,
                                    dropout=dropout,
                                    act_func=act_func,
                                    device=device),

            'ATGN' : TemporalGNN(in_channels=in_dim,
                                hidden_channels=64,
                                out_channels=64,
                                n_sequences=k_days,
                                device=device,
                                act_func=act_func,
                                dropout=dropout),

            'ST-GATLTSM' : ST_GATLSTM(in_channels=in_dim,
                                    hidden_channels=[64, 64, 64],
                                        out_channels=64,
                                        end_channels=32,
                                        n_sequences=k_days,
                                        device=device, act_func=act_func, heads=6, dropout=dropout),

            'Zhang' : Zhang(in_channels=in_dim_2D,
                        hidden_channels=64,
                            end_channels=128,
                            dropout=dropout,
                            binary=False,
                            device=device,
                            n_sequences=k_days),

            'ConvLSTM' : CONVLSTM(in_channels=in_dim_2D,
                                hidden_dim=[32, 32, 32],
                                    end_channels=64,
                                    n_sequences=k_days,
                                    dropout=dropout,
                                    device=device, act_func=act_func)
            }

/home/caron/Bureau/.conda/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
name = 'GAT'
display(dico_model[name])

GAT(
  (dropout): Dropout(p=0.03, inplace=False)
  (net): Sequential(
    (0) - GATConv(100, 64, heads=4): x, edge_index -> x
    (1) - ReLU(): x -> x
    (2) - GATConv(256, 64, heads=4): x, edge_index -> x
    (3) - ReLU(): x -> x
    (4) - GATConv(256, 64, heads=2): x, edge_index -> x
  )
  (output): OutputLayer(
    (fc): Linear(64, 64, bias=True)
    (activation): ReLU()
    (fc2): Linear(64, 1, bias=True)
    (output): ReLU()
  )
)